## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
LRG
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [131]:
save_filename='truth_table_Andes_reinspection_LRG_68001_20200315_2.csv'

In [132]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [133]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['68001']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
#del obs_db['67230-20200314']
#obs_db.keys()

In [134]:
pd.set_option('display.max_rows', 10)

In [135]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/LRG/'

In [136]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(4,7):
    pattern = "desi*_"+str(68001)+"_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_68001_4_*.csv
desi*_68001_5_*.csv
desi*_68001_6_*.csv
['desi-vi_LRG_reinspection_68001_4_PNT.csv', 'desi-vi_LRG_reinspection_68001_4_KSD.csv', 'desi-vi_LRG_reinspection_68001_5_PNT.csv', 'desi-vi_LRG_reinspection_68001_5_KSD.csv', 'desi-vi_LRG_reinspection_68001_6_PNT.csv', 'desi-vi_LRG_reinspection_68001_6_KSD.csv']


In [137]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_LRG_reinspection_68001_4_PNT.csv
desi-vi_LRG_reinspection_68001_4_KSD.csv
desi-vi_LRG_reinspection_68001_5_PNT.csv
desi-vi_LRG_reinspection_68001_5_KSD.csv
desi-vi_LRG_reinspection_68001_6_PNT.csv
desi-vi_LRG_reinspection_68001_6_KSD.csv
There are 299 visual inspections of a total of 150 unique objects
That gives 1.9933333333333334 inspections per object.


In [138]:
#vi is a dataframe
#display(vi)

In [139]:
#vi.keys()
len(vi)

299

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [140]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

5000
5000
299


In [141]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [142]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [143]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35191263070258916,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0160,PNT,...,179.033720,1.0160,GALAXY,4.0,0,0.000000,1.0160,--|--,2,none
1,35191263074452265,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.0811,PNT,...,25.988930,0.0811,GALAXY,3.0,2,0.433725,0.0811,Continuum and D1 and D2 detected but Halpha de...,2,none
2,35191263074453893,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9841,PNT,...,100.414572,0.9841,GALAXY,4.0,0,0.000000,0.9841,--|--,2,none
3,35191266727690614,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.7608,PNT,...,84.293783,0.7608,GALAXY,3.5,1,0.000000,0.7608,--|--,2,none
4,35191266727691301,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.7820,PNT,...,117.580083,0.7820,GALAXY,4.0,0,0.000000,0.7820,--|--,2,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,35191270393513564,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.2318,KSD,...,117.276860,1.2318,GALAXY,4.0,0,0.000000,1.2318,--|--,2,none
295,35191270393514617,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9112,KSD,...,20.049107,0.9112,GALAXY,3.0,2,0.000000,0.9112,--|--,2,none
296,35191270393515099,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.5158,KSD,...,3.677688,0.7473,GALAXY,2.0,0,0.439821,1.5158,Continuum detected But OII line and other line...,2,none
297,35191270397708396,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0958,KSD,...,187.587404,1.0958,GALAXY,4.0,0,0.000000,1.0958,--|--,2,none


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [144]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [145]:
len(vi_gp)

150

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [146]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191263074452265 35191263078646664 35191263082842426 35191263087037580
 35191266723497497 35191266740275158 35191266740275701 35191270385124281
 35191270393514617 35191273967059569 35191273975449926]
Total number of conflicts to resolve:  11


In [147]:
unique_targets[0]

35191263074452265

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [148]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [149]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [150]:
show_spectra = False

In [151]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
1,35191263074452265,GALAXY,0.0811,PNT,2,RS,0.0811,GALAXY,Continuum and D1 and D2 detected but Halpha de...,0.0811,GALAXY,3.0,2,0.0811,0.433725,Continuum and D1 and D2 detected but Halpha de...,2,none
51,35191263074452265,GALAXY,0.0811,KSD,4,R,0.5500,NaN,possible blend,0.5500,GALAXY,3.0,2,0.0811,0.433725,Continuum and D1 and D2 detected but Halpha de...,2,none


In [152]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5500
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Redshif is robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

1     4
51    4
Name: VI_quality, dtype: int64
conflict ID =  0


In [153]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
40,35191263078646664,GALAXY,0.8768,PNT,4,NaN,0.8768,GALAXY,--,0.8768,GALAXY,3.0,2,0.8768,0.0,--|--,2,none
90,35191263078646664,GALAXY,0.8768,KSD,2,NaN,0.8768,NaN,--,0.8768,GALAXY,3.0,2,0.8768,0.0,--|--,2,none


In [154]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8101
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Possible weak CaII. Be conservative.'
##look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



conflict ID =  1


In [155]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
48,35191263082842426,GALAXY,0.9126,PNT,4,NaN,0.9126,GALAXY,--,0.9126,GALAXY,3.0,2,0.9126,0.0,--|--,2,none
98,35191263082842426,GALAXY,0.9126,KSD,2,NaN,0.9126,NaN,--,0.9126,GALAXY,3.0,2,0.9126,0.0,--|--,2,none


In [156]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2320
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Low S/N CaII. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  2


In [157]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
23,35191263087037580,GALAXY,0.601,PNT,2,S,0.601,GALAXY,Ca(K) is missing Hdelta is detected as absorption,0.601,GALAXY,3.0,2,0.601,0.0,Ca(K) is missing Hdelta is detected as absorpt...,2,none
73,35191263087037580,GALAXY,0.601,KSD,4,NaN,0.601,NaN,--,0.601,GALAXY,3.0,2,0.601,0.0,Ca(K) is missing Hdelta is detected as absorpt...,2,none


In [158]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


conflict ID =  3


In [159]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
7,35191266723497497,GALAXY,0.0255,PNT,2,RS,NaN,GALAXY,Seems like an LRG at small redshift Ha and NII...,0.0255,GALAXY,3.0,2,0.0255,1.345685,Seems like an LRG at small redshift Ha and NII...,2,none
57,35191266723497497,GALAXY,0.0255,KSD,4,NaN,1.4055,QSO,--,1.4055,QSO,3.0,2,0.0255,1.345685,Seems like an LRG at small redshift Ha and NII...,2,none


In [160]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.4055
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is robust. red QSO'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

conflict ID =  4


In [161]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
123,35191266740275158,GALAXY,0.9632,PNT,4,NaN,0.9632,GALAXY,--,0.9632,GALAXY,3.0,2,0.9632,0.0,--|--,2,none
173,35191266740275158,GALAXY,0.9632,KSD,2,NaN,0.9632,NaN,--,0.9632,GALAXY,3.0,2,0.9632,0.0,--|--,2,none


In [162]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Low S/N CaII. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  5


In [163]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
125,35191266740275701,GALAXY,0.9608,PNT,4,NaN,0.9608,GALAXY,--,0.9608,GALAXY,3.0,2,0.9608,0.0,--|--,2,none
175,35191266740275701,GALAXY,0.9608,KSD,2,NaN,0.9608,NaN,--,0.9608,GALAXY,3.0,2,0.9608,0.0,--|--,2,none


In [164]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Low S/N Possible CaII and G band'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  6


In [165]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
229,35191270385124281,GALAXY,0.7658,PNT,4,NaN,0.7658,GALAXY,--,0.7658,GALAXY,3.0,2,0.7658,0.0,--|--,2,none
279,35191270385124281,GALAXY,0.7658,KSD,2,NaN,0.7658,NaN,--,0.7658,GALAXY,3.0,2,0.7658,0.0,--|--,2,none


In [166]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1847
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Reshift should be secure'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  7


In [167]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
245,35191270393514617,GALAXY,0.9112,PNT,4,NaN,0.9112,GALAXY,--,0.9112,GALAXY,3.0,2,0.9112,0.0,--|--,2,none
295,35191270393514617,GALAXY,0.9112,KSD,2,NaN,0.9112,NaN,--,0.9112,GALAXY,3.0,2,0.9112,0.0,--|--,2,none


In [168]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1920
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = "CaII in bad spectral region. Be conservative."

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  8


In [169]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
205,35191273967059569,GALAXY,0.1156,PNT,4,NaN,0.1156,GALAXY,--,0.1156,GALAXY,4.0,0,0.1156,0.324668,--|Two objects in spectrum (z=0.11 + 0.47),2,none
254,35191273967059569,GALAXY,0.1156,KSD,4,NaN,0.4778,NaN,Two objects in spectrum (z=0.11 + 0.47),0.4778,GALAXY,4.0,0,0.1156,0.324668,--|Two objects in spectrum (z=0.11 + 0.47),2,none


In [170]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.4778
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Two objects z=0.1156 and z=0.4778'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  9


In [171]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
210,35191273975449926,QSO,0.8088,PNT,4,NaN,0.8088,GALAXY,--,0.8088,GALAXY,4.0,0,0.809,0.000221,--|--,2,none
260,35191273975449926,QSO,0.8088,KSD,4,NaN,0.8092,NaN,--,0.8092,QSO,4.0,0,0.809,0.000221,--|--,2,none


In [172]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8092
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is robust'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  10


### and so on...

We should now recompute the conflicts, and not find any.

In [173]:
vi_conflict = VI.find_conflicts(vi_gp)

In [174]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [175]:
vi_conflict = VI.find_conflicts(vi_gp)

In [176]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [177]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [178]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [179]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/truth_table_Andes_reinspection_LRG_68001_20200315_2.csv


In [180]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-180-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [181]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [182]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [183]:
len(merged_file)

150

In [184]:
VI_dir

'/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/'